Nina Robbins:

Import modules

In [ ]:
import astropy
import sep
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import ascii

rcParams['figure.figsize'] = [16., 13.]

Open and visualize the fits file

In [ ]:
fname = 'hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits'
hdu_list = fits.open(fname)
#image_head = hdu_list[0].header
#print(image_head)

In [ ]:
image_data = hdu_list[0].data
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation = 'nearest', cmap='gray', vmin = m-s, vmax = m+s, origin='lower')
plt.colorbar()

Measure a spatially varying background on image


In [ ]:
image_data = image_data.byteswap().newbyteorder()
bkg = sep.Background(image_data)
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back() #evaluates background as a 2d array, like the original image
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')

Background noise as a 2d array

In [ ]:
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')

Subtract the background

In [ ]:
data_sub = image_data-bkg
im = plt.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

Object detetction

In [ ]:
thresh = 4 #we make the threshold higher to be able to make it easier to distinguish from noise
objects = sep.extract(data_sub, thresh, err=bkg.globalrms)
print('Number of objects detected: ', len(objects))

Show background-subtracted image and plot an ellipse around each detected object

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=6*objects['a'][i], height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
plt.savefig('f105w_source_detection.png')